In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from datasets import load_dataset

dataset = load_dataset("society-ethics/lila_camera_traps", "Caltech Camera Traps")


Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file_name', 'width', 'height', 'seq_num_frames', 'date_captured', 'seq_id', 'location', 'rights_holder', 'frame_num', 'annotations', 'bboxes', 'image'],
        num_rows: 243100
    })
})

In [5]:
import cv2
from skimage.feature import local_binary_pattern
from skimage import io
import numpy as np
from tqdm import tqdm
from datasets import load_dataset

In [6]:
def extract_color_histogram(image):
    hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

col histograms

In [7]:
def extract_lbp(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, 8, 1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 64), range=(0, 256))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    return hist


local binary pattern - texture

In [8]:
def extract_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return edges.flatten()

canny algo - edges

In [9]:
def extract_shape_descriptors(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return [0]  
    
    contour_area = [cv2.contourArea(contour) for contour in contours]
    return contour_area


contour areas

In [10]:
import requests
import numpy as np
from tqdm import tqdm
import cv2

**taking 1 from 100 values because 231400 is actual dataset**

In [ ]:
# import requests
# import numpy as np
# from tqdm import tqdm
# import cv2
# i=0

# all_features = []

# for example in tqdm(dataset["train"]):
   
#     try:
#         # Extract the image URL from the example
#         image_url = example["image"]
        
#         # Download the image from the URL
#         response = requests.get(image_url)
        
#         # Check if the response is successful
#         if response.status_code == 200:
#             # Decode the image
#             image_array = np.frombuffer(response.content, np.uint8)
#             image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
            
#             # Check if the image decoding was successful
#             if image is not None:
#                 # Extract features
#                 color_hist_features = extract_color_histogram(image)
#                 lbp_features = extract_lbp(image)
#                 edge_features = extract_edges(image)
#                 shape_features = extract_shape_descriptors(image)
                
#                 # Combine features
#                 combined_features = np.concatenate([color_hist_features, lbp_features, edge_features, shape_features])
#                 all_features.append(combined_features)
#             else:
#                 print("Error: Unable to decode image.")
#         else:
#             print(f"Error: Unable to download image - Status Code: {response.status_code}")
#     except Exception as e:
#         print(f"Error occurred while processing image: {e}")


In [11]:
import numpy as np
all_labels = []
for i, example in enumerate(tqdm(dataset["train"])):
  
        try:
            
            category_name = example["annotations"]["taxonomy"][0]["species"] 
            all_labels.append(category_name)
        except Exception as e:
            print(f"Error occurred while processing image: {e}")

all_labels = np.array(all_labels)
print(all_labels)


100%|██████████| 243100/243100 [00:56<00:00, 4284.28it/s]

[None None 2 ... None None 8]


In [12]:
non_none_labels = all_labels[all_labels != None]
print(non_none_labels)
print(len(non_none_labels))

[2 8 5 ... 8 5 8]
67692


In [13]:
all_labels = np.array(all_labels)
print(all_labels)

[None None 2 ... None None 8]


In [14]:
import pandas as pd

# Assuming all_features, all_labels2, and all_labels are already populated
# Assuming extract_color_histogram(), extract_lbp(), extract_edges(), and extract_shape_descriptors() functions are defined

sampling_interval = 100

color_hist_features_list = []
lbp_features_list = []
edge_features_list = []
shape_features_list = []
labels_list = []

for i, (example, label) in enumerate(zip(tqdm(dataset["train"]), all_labels)):
    if (i > 121550):
        break
    if i % sampling_interval == 0 and label is not None:
        try:
            image_url = example["image"]
            response = requests.get(image_url)

            if response.status_code == 200:
                image_array = np.frombuffer(response.content, np.uint8)
                image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

                if image is not None:
                    color_hist_features = extract_color_histogram(image)
                    lbp_features = extract_lbp(image)
                    edge_features = extract_edges(image)
                    shape_features = extract_shape_descriptors(image)

                    color_hist_features_list.append(color_hist_features)
                    lbp_features_list.append(lbp_features)
                    edge_features_list.append(edge_features)
                    shape_features_list.append(shape_features)
                    labels_list.append(label)
                else:
                    print("Error: Unable to decode image.")
            else:
                print(f"Error: Unable to download image - Status Code: {response.status_code}")
        except Exception as e:
            print(f"Error occurred while processing image: {e}")

# Create a dataframe
df = pd.DataFrame({
    'color_hist_features': color_hist_features_list,
    'lbp_features': lbp_features_list,
    'edge_features': edge_features_list,
    'shape_features': shape_features_list,
    'labels': labels_list
})

# Save the dataframe to a CSV file
df.to_csv('features_and_labels.csv', index=False)

print("Dataframe saved successfully.")


 50%|█████     | 121551/243100 [11:27<11:27, 176.70it/s] 


Dataframe saved successfully.


In [13]:
# import requests
# import numpy as np
# from tqdm import tqdm
# import cv2


# sampling_interval = 100

# all_features = []

# all_labels2=[]

# for i, (example, label) in enumerate(zip(tqdm(dataset["train"]), all_labels)):
#     if (i>121550):
#         break
#     if i % sampling_interval == 0 and label is not None:
#         try:
           
#             image_url = example["image"]

            
#             response = requests.get(image_url)

           
#             if response.status_code == 200:
                
#                 image_array = np.frombuffer(response.content, np.uint8)
#                 image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

                
#                 if image is not None:
                    
#                     color_hist_features = extract_color_histogram(image)
#                     lbp_features = extract_lbp(image)
#                     edge_features = extract_edges(image)
#                     shape_features = extract_shape_descriptors(image)

#                     combined_features = np.concatenate([color_hist_features, lbp_features, edge_features, shape_features])
#                     all_features.append(combined_features)
#                     all_labels2.append(label)
#                 else:
#                     print("Error: Unable to decode image.")
#             else:
#                 print(f"Error: Unable to download image - Status Code: {response.status_code}")
#         except Exception as e:
#             print(f"Error occurred while processing image: {e}")


# all_features = np.array(all_features)
# all_labels = np.array(all_labels)


 50%|█████     | 121551/243100 [07:11<07:11, 281.84it/s] 


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (309,) + inhomogeneous part.

In [ ]:
import numpy as np

np.save('all_features.npy', [arr.tolist() for arr in all_features[:308]])
np.save('all_labels.npy', all_labels2[:308])

print("Arrays saved successfully.")

In [16]:
import pandas as pd
df_features = pd.DataFrame(all_features)
df_features

KeyboardInterrupt: 

In [ ]:
df_features

In [16]:
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

# Load pre-trained CNN model without top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Function to preprocess and extract features from an image
import tempfile
import os

def extract_features(image_array):
    # Save the image array to a temporary file
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
    temp_file_path = temp_file.name
    cv2.imwrite(temp_file_path, cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB))
    temp_file.close()
    
    # Load and preprocess the image
    img = load_img(temp_file_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize pixel values
    
    # Extract features using the pre-trained CNN model
    features = base_model.predict(img_array)
    
    # Flatten the features
    flattened_features = features.flatten()
    
    # Delete the temporary file
    os.unlink(temp_file_path)
    
    return flattened_features

# Assuming all_labels2 and all_labels are already populated
sampling_interval = 100

features_list = []
labels_list = []

for i, (example, label) in enumerate(zip(tqdm(dataset["train"]), all_labels)):
    if (i > 121550):
        break
    if i % sampling_interval == 0 and label is not None:
        try:
            image_url = example["image"]
            response = requests.get(image_url)

            if response.status_code == 200:
                image_array = np.frombuffer(response.content, np.uint8)
                image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

                if image is not None:
                    features = extract_features(image)
                    features_list.append(features)
                    labels_list.append(label)
                else:
                    print("Error: Unable to decode image.")
            else:
                print(f"Error: Unable to download image - Status Code: {response.status_code}")
        except Exception as e:
            print(f"Error occurred while processing image: {e}")

# Convert the feature and label lists to NumPy arrays
X = np.array(features_list)
y = np.array(labels_list)



  0%|          | 0/243100 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


I0000 00:00:1714547875.281291      91 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
  0%|          | 201/243100 [00:06<2:09:09, 31.34it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


  0%|          | 501/243100 [00:08<56:35, 71.45it/s]  

...

 50%|████▉     | 120954/243100 [08:27<02:16, 897.74it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


 50%|████▉     | 121301/243100 [08:28<04:25, 458.75it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


 50%|█████     | 121551/243100 [08:30<08:30, 238.08it/s]


In [18]:
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

In [19]:
y


array([ 8,  8,  8, 10,  5,  5,  8,  8, 10, 10,  8,  8,  8,  2,  2,  8,  5,
       10, 10,  1,  8,  4,  8,  8,  8,  8,  4,  1,  4,  4,  8,  8,  2,  1,
       10,  4,  8,  4,  4, 10,  1,  8, 10,  2,  1,  1,  1,  4, 10,  8,  6,
        8,  2,  4,  4,  4,  8,  8,  8,  4,  4,  8,  2,  8,  1, 10,  8,  2,
       10, 10,  1, 10,  4,  8, 10, 10,  4, 10,  4,  8,  6,  2, 10,  8, 10,
        4,  8,  4,  4,  8,  1,  1,  8,  5,  8,  8, 10,  8,  8,  8,  2,  1,
        6,  8,  1,  8,  8,  4,  4,  8,  1,  2,  8,  6, 10,  1,  8,  2,  8,
        6,  8,  4,  8,  8,  1,  8,  1, 10, 10,  1,  4,  2,  8,  8,  8,  2,
        4,  5,  1,  8,  6,  8,  8,  8,  8, 10,  2,  8,  1,  5,  1,  6,  8,
        1,  8,  8, 10,  8,  4,  8,  8,  1,  1,  1,  8,  8,  4,  4,  8,  8,
        2,  1,  8,  1,  5,  8, 10,  4,  4,  8, 10,  2,  4,  4,  2, 10,  8,
        8,  4,  4,  2, 10,  8,  5,  4,  4,  8,  4,  5, 10,  8,  8,  4,  8,
        5, 10, 10,  5, 10,  4,  6,  5,  6,  2,  8,  8,  8,  4,  4,  4,  8,
        4,  8, 10,  8,  8

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
len(X[2])

25088

In [27]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)


accuracy = svm_classifier.score(X_valid, y_valid)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5483870967741935


In [28]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# Initialize the SVM classifier
svm = SVC()

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best estimator
best_svm = grid_search.best_estimator_

# Evaluate the model
accuracy = best_svm.score(X_test, y_test)
print("Best Parameters:", grid_search.best_params_)
print("Accuracy:", accuracy)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   1.0s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   1.6s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   2.6s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   1.8s
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time=   1.5s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=   1.8s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=   1.9s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   2.6s
[CV] END ......................C=0.1, gamma=0.1, kernel=poly; total time=   1.8s
[CV] END ...................C=0.1, gamma=0.1, kernel=sigmoid; total time=   1.4s
[CV] END ...................C=0.1, gamma=0.01, kernel=linear; total time=   1.7s
[CV] END ...................C=0.1, gamma=0.01, 